In [1]:
from bs4 import BeautifulSoup as bs
import requests
import folium
import re

In [2]:
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        return getLatLng(fixed_addr)

def add_marker(mapobj, x, y, info):
    folium.Marker((y,x), popup=(info[0] + '\n' + info[1]), icon=folium.Icon(color='red', icon='info-sign')).add_to(mapobj)

def get_avg_point(point_list):
    x = sum(map(lambda i:float(i[0]), point_list)) / len(point_list)
    y = sum(map(lambda i:float(i[1]), point_list)) / len(point_list)
    return y,x

In [3]:
# load html
url = 'https://www.wikitree.co.kr/articles/217101'
resp = requests.get(url)

# dataset
soup = bs(resp.text.replace('&nbsp;', ' '),'html.parser')
news_content = soup.select_one('div#wikicon')
store_list = []
tel_list = []
addr_list = []
stores = list(map(lambda i : i.text.split('회 '), news_content.select('strong')))
tel_p = re.compile('^\d{2,3}-\d{3,4}-\d{4}')
addr_p = re.compile('^[가-힣 ]{0,4}시?[가-힣]{1,3}구 [가-힣]{0,3}동?[0-9 ]')
addr_err = re.compile('^동[0-9 ]{0,3}-?[0-9]{0,2}')
# create each list
for store in stores:
    if len(store) > 1:
        store_list.append(store[-1])
tel_list = news_content.find_all(string = tel_p)
addr_list = news_content.find_all(string = addr_p)
store_total = [(store.strip(), tel.strip(), addr.strip()) for store,tel,addr in zip(store_list, tel_list, addr_list)]

# test 
addr_point_list = list(map(lambda i: getLatLng(i[2]),store_total))

In [17]:
map_food = folium.Map(location = get_avg_point(addr_point_list), zoom_start=11)
cnt = 0
for x,y in addr_point_list:
    add_marker(map_food, x, y, store_total[cnt])
    cnt += 1
map_food 

In [4]:
addr_list[16]

'종로구 당주동 5 로얄빌딩 지하 1층'